In [8]:
import sklearn
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
import scipy
import torch

In [ ]:
from sklearn.model_selection import train_test_split

In [113]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample = pd.read_csv("data/sample_submission.csv")


In [114]:
train.head(5)

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,140,1358,0,1,50,0.7393,0.4000,0.5000,0.0059,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,111,1687,1,0,80,0.7772,0.2878,0.2581,0.0044,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,141,1400,0,1,40,0.0557,0.5282,0.9895,0.1077,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,134,1387,0,1,40,0.7202,0.3333,0.3333,0.0044,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,111,1692,0,1,300,0.1211,0.5347,0.0842,0.0192,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [115]:
train.drop("id",inplace=True,axis=1)
test.drop("id",inplace=True,axis=1)
id = sample["id"]
names = sample.columns.tolist()
sample.drop("id",inplace=True,axis=1)


In [116]:
y_col = sample.columns.tolist()

y_train = train[y_col]
X_train = train.drop(y_col,axis=1)

print(X_train.shape)
print(y_train.shape)

(19219, 27)
(19219, 7)


In [117]:
X_train.head(10)

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,584,590,909972,909977,16,8,5,2274,113,140,1358,0,1,50,0.7393,0.4000,0.5000,0.0059,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,-0.5000,-0.0104,0.1417
1,808,816,728350,728372,433,20,54,44478,70,111,1687,1,0,80,0.7772,0.2878,0.2581,0.0044,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,-0.2997,0.9491
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,1400,0,1,40,0.0557,0.5282,0.9895,0.1077,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,-0.0105,-0.0944,1.0000
3,781,789,3353146,3353173,210,16,29,3202,114,134,1387,0,1,40,0.7202,0.3333,0.3333,0.0044,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,-0.0402,0.4025
4,1540,1560,618457,618502,521,72,67,48231,82,111,1692,0,1,300,0.1211,0.5347,0.0842,0.0192,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,-0.2455,0.9998
5,1009,1033,899231,899307,409,22,26,47513,86,118,1650,1,0,70,0.2761,0.4136,0.4091,0.0053,0.5454,1.0000,1.0,2.6117,0.9542,1.4150,0.5909,-0.1890,0.8749
6,596,607,739072,7390760,204,21,21,22478,89,127,1373,0,1,40,0.8331,0.2744,0.6842,0.0110,0.7143,1.0000,1.0,2.3096,1.1761,1.3222,0.3158,-0.1497,0.5212
7,1673,1687,294065,294091,571,38,57,53142,77,110,1692,0,1,300,0.1491,0.4326,0.9643,0.0142,0.5686,0.7179,1.0,2.7528,1.3802,1.7559,0.0357,-0.2661,0.9408
8,507,521,203252,203261,101,25,12,12530,124,140,1360,0,1,40,0.7210,0.4815,0.3333,0.0176,0.8000,0.7500,0.0,2.0043,1.3802,0.7782,-0.6667,0.0305,0.3601
9,893,907,1341292,1341296,54,6,15,7052,87,133,1687,1,0,60,0.8088,0.2000,0.5333,0.0037,0.8333,1.0000,1.0,1.7324,0.6990,1.1761,0.4667,-0.1228,0.1400


In [49]:
x = torch.Tensor(X_train.values)
y = torch.Tensor(y_train.values)

In [155]:
device = torch.device('cpu')
N, D_in,  D_out = 19219, 27, 7
H = [500,1000,500,100]

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H[0]),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H[0], H[1]),
    torch.nn.ReLU(),
    torch.nn.Linear(H[1], H[2]),
    torch.nn.ReLU(),
    torch.nn.Linear(H[2], D_out)
)

l_r = 1e-10
for i in range(10):
    y_pred = model(x)
    loss = torch.nn.MSELoss(y_pred,y)
    print(loss)
    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= l_r*param.grad
    model.zero_grad()

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [182]:
y1 = y[:,1].reshape(19219, 1)

In [211]:
device = torch.device('cpu')
N, D_in,  D_out = 19219, 27, 1
H = [500,1000,500,100]

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H[0]),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H[0], H[1]),
    torch.nn.ReLU(),
    torch.nn.Linear(H[1], H[2]),
    torch.nn.ReLU(),
    torch.nn.Linear(H[2], D_out)#,
 #   torch.nn.Softmax()
    
)

l_r = 1e-2
for i in range(200):
    l_r  = 1e-9/(i+1)
    y_pred = model(x)
    #loss = torch.nn.functional.binary_cross_entropy_with_logits(y_pred,y1)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(y_pred,y)
    print(loss)
    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= l_r*param.grad
    model.zero_grad()

tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1.2534, grad_fn=<BinaryCrossEntro

In [212]:
X_test_2l = torch.Tensor(test.values)
y_pred_2l = model(X_test_2l)
torch.nn.functional.softmax(y_pred_2l)

<ipython-input-212-6ed6405aa3d4>:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(y_pred_2l)


tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]], grad_fn=<SoftmaxBackward0>)

In [126]:
y_pred_2l = pd.DataFrame(y_pred_2l).astype("float")
y_pred_2l.columns = sample.columns
y_pred_2l["id"] = id


In [127]:
y_pred_2l = y_pred_2l[names]
y_pred_2l.to_csv("output/2lnn_1.csv",index=False)